In [20]:
import pandas as pd
import numpy as np
from collections import defaultdict
pd.set_option('display.max_columns', None)

In [21]:
# Writer which will be used until the end
writer = pd.ExcelWriter("tmp_ajt_processed_data.xlsx", engine='xlsxwriter')

In [22]:
# Load the data into a df and remove the junk headers
df = pd.read_csv("Adverb++Locations_February+20,+2021_06.38.csv")
df = df.iloc[2:]
df.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,SCT Trial_0_GROUP,SCT Trial_1_GROUP,SCT Trial_2_GROUP,SCT Trial_3_GROUP,SCT Trial_0_1_RANK,SCT Trial_0_2_RANK,SCT Trial_0_3_RANK,SCT Trial_0_11_RANK,SCT Trial_1_1_RANK,SCT Trial_1_2_RANK,SCT Trial_1_3_RANK,SCT Trial_1_11_RANK,SCT Trial_2_1_RANK,SCT Trial_2_2_RANK,SCT Trial_2_3_RANK,SCT Trial_2_11_RANK,SCT Trial_3_1_RANK,SCT Trial_3_2_RANK,SCT Trial_3_3_RANK,SCT Trial_3_11_RANK,SCT Trial 2_0_GROUP,SCT Trial 2_1_GROUP,SCT Trial 2_2_GROUP,SCT Trial 2_3_GROUP,SCT Trial 2_0_1_RANK,SCT Trial 2_0_2_RANK,SCT Trial 2_0_3_RANK,SCT Trial 2_0_11_RANK,SCT Trial 2_1_1_RANK,SCT Trial 2_1_2_RANK,SCT Trial 2_1_3_RANK,SCT Trial 2_1_11_RANK,SCT Trial 2_2_1_RANK,SCT Trial 2_2_2_RANK,SCT Trial 2_2_3_RANK,SCT Trial 2_2_11_RANK,SCT Trial 2_3_1_RANK,SCT Trial 2_3_2_RANK,SCT Trial 2_3_3_RANK,SCT Trial 2_3_11_RANK,SCT0_A-_aqui_0_GROUP,SCT0_A-_aqui_1_GROUP,SCT0_A-_aqui_2_GROUP,SCT0_A-_aqui_3_GROUP,SCT0_A-_aqui_0_1_RANK,SCT0_A-_aqui_0_2_RANK,SCT0_A-_aqui_0_3_RANK,SCT0_A-_aqui_0_4_RANK,SCT0_A-_aqui_1_1_RANK,SCT0_A-_aqui_1_2_RANK,SCT0_A-_aqui_1_3_RANK,SCT0_A-_aqui_1_4_RANK,SCT0_A-_aqui_2_1_RANK,SCT0_A-_aqui_2_2_RANK,SCT0_A-_aqui_2_3_RANK,SCT0_A-_aqui_2_4_RANK,SCT0_A-_aqui_3_1_RANK,SCT0_A-_aqui_3_2_RANK,SCT0_A-_aqui_3_3_RANK,SCT0_A-_aqui_3_4_RANK,SCT1_A-_aqui_0_GROUP,SCT1_A-_aqui_1_GROUP,SCT1_A-_aqui_2_GROUP,SCT1_A-_aqui_3_GROUP,SCT1_A-_aqui_0_1_RANK,SCT1_A-_aqui_0_2_RANK,SCT1_A-_aqui_0_3_RANK,SCT1_A-_aqui_0_4_RANK,SCT1_A-_aqui_1_1_RANK,SCT1_A-_aqui_1_2_RANK,SCT1_A-_aqui_1_3_RANK,SCT1_A-_aqui_1_4_RANK,SCT1_A-_aqui_2_1_RANK,SCT1_A-_aqui_2_2_RANK,SCT1_A-_aqui_2_3_RANK,SCT1_A-_aqui_2_4_RANK,SCT1_A-_aqui_3_1_RANK,SCT1_A-_aqui_3_2_RANK,SCT1_A-_aqui_3_3_RANK,SCT1_A-_aqui_3_4_RANK,SCT0_A-_afuera_0_GROUP,SCT0_A-_afuera_1_GROUP,SCT0_A-_afuera_2_GROUP,SCT0_A-_afuera_3_GROUP,SCT0_A-_afuera_0_1_RANK,SCT0_A-_afuera_0_2_RANK,SCT0_A-_afuera_0_3_RANK,SCT0_A-_afuera_0_4_RANK,SCT0_A-_afuera_1_1_RANK,SCT0_A-_afuera_1_2_RANK,SCT0_A-_afuera_1_3_RANK,SCT0_A-_afuera_1_4_RANK,SCT0_A-_afuera_2_1_RANK,SCT0_A-_afuera_2_2_RANK,SCT0_A-_afuera_2_3_RANK,SCT0_A-_afuera_2_4_RANK,SCT0_A-_afuera_3_1_RANK,SCT0_A-_afuera_3_2_RANK,SCT0_A-_afuera_3_3_RANK,SCT0_A-_afuera_3_4_RANK,SCT1_A-_afuera_0_GROUP,SCT1_A-_afuera_1_GROUP,SCT1_A-_afuera_2_GROUP,SCT1_A-_afuera_3_GROUP,SCT1_A-_afuera_0_1_RANK,SCT1_A-_afuera_0_2_RANK,SCT1_A-_afuera_0_3_RANK,SCT1_A-_afuera_0_4_RANK,SCT1_A-_afuera_1_1_RANK,SCT1_A-_afuera_1_2_RANK,SCT1_A-_afuera_1_3_RANK,SCT1_A-_afuera_1_4_RANK,SCT1_A-_afuera_2_1_RANK,SCT1_A-_afuera_2_2_RANK,SCT1_A-_afuera_2_3_RANK,SCT1_A-_afuera_2_4_RANK,SCT1_A-_afuera_3_1_RANK,SCT1_A-_afuera_3_2_RANK,SCT1_A-_afuera_3_3_RANK,SCT1_A-_afuera_3_4_RANK,SCT0_A-_cerca_0_GROUP,SCT0_A-_cerca_1_GROUP,SCT0_A-_cerca_2_GROUP,SCT0_A-_cerca_3_GROUP,SCT0_A-_cerca_0_1_RANK,SCT0_A-_cerca_0_2_RANK,SCT0_A-_cerca_0_3_RANK,SCT0_A-_cerca_0_4_RANK,SCT0_A-_cerca_1_1_RANK,SCT0_A-_cerca_1_2_RANK,SCT0_A-_cerca_1_3_RANK,SCT0_A-_cerca_1_4_RANK,SCT0_A-_cerca_2_1_RANK,SCT0_A-_cerca_2_2_RANK,SCT0_A-_cerca_2_3_RANK,SCT0_A-_cerca_2_4_RANK,SCT0_A-_cerca_3_1_RANK,SCT0_A-_cerca_3_2_RANK,SCT0_A-_cerca_3_3_RANK,SCT0_A-_cerca_3_4_RANK,SCT1_A-_cerca_0_GROUP,SCT1_A-_cerca_1_GROUP,SCT1_A-_cerca_2_GROUP,SCT1_A-_cerca_3_GROUP,SCT1_A-_cerca_0_1_RANK,SCT1_A-_cerca_0_2_RANK,SCT1_A-_cerca_0_3_RANK,SCT1_A-_cerca_0_4_RANK,SCT1_A-_cerca_1_1_RANK,SCT1_A-_cerca_1_2_RANK,SCT1_A-_cerca_1_3_RANK,SCT1_A-_cerca_1_4_RANK,SCT1_A-_cerca_2_1_RANK,SCT1_A-_cerca_2_2_RANK,SCT1_A-_cerca_2_3_RANK,SCT1_A-_cerca_2_4_RANK,SCT1_A-_cerca_3_1_RANK,SCT1_A-_cerca_3_2_RANK,SCT1_A-_cerca_3_3_RANK,SCT1_A-_cerca_3_4_RANK,SCT0_A-_lejos_0_GROUP,SCT0_A-_lejos_1_GROUP,SCT0_A-_lejos_2_GROUP,SCT0_A-_lejos_3_GROUP,SCT0_A-_lejos_0_1_RANK,SCT0_A-_lejos_0_2_RANK,SCT0_A-_lejos_0_3_RANK,SCT0_A-_lejos_0_4_RANK,SCT0_A-_lejos_1_1_RANK,SCT0_A-_lejos_1_2_RANK,

In [23]:
# Remove people who did not finish the experiment and convert all the columns to numeric when possible
# I am doing this by grabbing everyone who had a progress of 100 or 99 because there are some people who finished
# but the progress was not 100. Weird.
finished_df = df[(df['Progress'] == '100') | (df['Progress'] == '99')]
finished_df = finished_df[finished_df["Drop"] != 'x']
finished_df = finished_df.apply(pd.to_numeric, errors='ignore')

In [24]:
# Make a df for the learners and remove the test "participant"
learners_df = finished_df[ (finished_df["N1"].isnull() ) & ( finished_df["L10"] != "TEST" )]

# Also make a df for the natives
natives_df = finished_df[finished_df["N1"].notnull()]

In [25]:
# Lets get some basic statistics for each question for the learners and native speakers
filter_cols = [col for col in learners_df if "AJT" in col and "trial" not in col.lower()]
learners_ajt_df = learners_df[filter_cols]
learners_ajt_df.describe().to_excel(writer, sheet_name="learners_overview", index=True)
learners_ajt_df.describe()

,AJT0_AA-_aqui,AJT0_BA-_aqui,AJT0_CA-_aqui,AJT0_DA-_aqui,AJT0_AA-_afuera,AJT0_BA-_afuera,AJT0_CA-_afuera,AJT0_DA-_afuera,AJT0_AA-_cerca,AJT0_BA-_cerca,AJT0_CA-_cerca,AJT0_DA-_cerca,AJT0_AA-_lejos,AJT0_BA-_lejos,AJT0_CA-_lejos,AJT0_DA-_lejos,AJT0_AB-_casi,AJT0_BB-_casi,AJT0_CB-_casi,AJT0_DB-_casi,AJT0_AB+_aproximadamente,AJT0_BB+_aproximadamente,AJT0_CB+_aproximadamente,AJT0_DB+_aproximadamente,AJT0_AB-_apenas,AJT0_BB-_apenas,AJT0_CB-_apenas,AJT0_DB-_apenas,AJT0_AB+_solamente,AJT0_BB+_solamente,AJT0_CB+_solamente,AJT0_DB+_solamente,AJT0_AC+_facilmente,AJT0_BC+_facilmente,AJT0_CC+_facilmente,AJT0_DC+_facilmente,AJT0_AC-_bien,AJT0_BC-_bien,AJT0_CC-_bien,AJT0_DC-_bien,AJT0_AC-_mal,AJT0_BC-_mal,AJT0_CC-_mal,AJT0_DC-_mal,AJT0_AC+_precisamente,AJT0_BC+_precisamente,AJT0_CC+_precisamente,AJT0_DC+_precisamente,AJT0_AD-_mucho,AJT0_BD-_mucho,AJT0_CD-_mucho,AJT0_DD-_mucho,AJT0_AD+_mayormente,AJT0_BD+_mayormente,AJT0_CD+_mayormente,AJT0_DD+_mayormente,AJT0_AD-_poco,AJT0_BD-_poco,AJT0_CD-_poco,AJT0_DD-_poco,AJT0_AD+_minimamente,AJT0_BD+_minimamente,AJT0_CD+_minimamente,AJT0_DD+_minimamente,AJT1_AA-_aqui,AJT1_BA-_aqui,AJT1_CA-_aqui,AJT1_DA-_aqui,AJT1_AA-_afuera,AJT1_BA-_afuera,AJT1_CA-_afuera,AJT1_DA-_afuera,AJT1_AA-_cerca,AJT1_BA-_cerca,AJT1_CA-_cerca,AJT1_DA-_cerca,AJT1_AA-_lejos,AJT1_BA-_lejos,AJT1_CA-_lejos,AJT1_DA-_lejos,AJT1_AB-_casi,AJT1_BB-_casi,AJT1_CB-_casi,AJT1_DB-_casi,AJT1_AB+_aproximadamente,AJT1_BB+_aproximadamente,AJT1_CB+_aproximadamente,AJT1_DB+_aproximadamente,AJT1_AB-_apenas,AJT1_BB-_apenas,AJT1_CB-_apenas,AJT1_DB-_apenas,AJT1_AB+_solamente,AJT1_BB+_solamente,AJT1_CB+_solamente,AJT1_DB+_solamente,AJT1_AC+_facilmente,AJT1_BC+_facilmente,AJT1_CC+_facilmente,AJT1_DC+_facilmente,AJT1_AC-_bien,AJT1_BC-_bien,AJT1_CC-_bien,AJT1_DC-_bien,AJT1_AC-_mal,AJT1_BC-_mal,AJT1_CC-_mal,AJT1_DC-_mal,AJT1_AC+_precisamente,AJT1_BC+_precisamente,AJT1_CC+_precisamente,AJT1_DC+_precisamente,AJT1_AD-_mucho,AJT1_BD-_mucho,AJT1_CD-_mucho,AJT1_DD-_mucho,AJT1_AD+_mayormente,AJT1_BD+_mayormente,AJT1_CD+_mayormente,AJT1_DD+_mayormente,AJT1_AD-_poco,AJT1_BD-_poco,AJT1_CD-_poco,AJT1_DD-_poco,AJT1_AD+_minimamente,AJT1_BD+_minimamente,AJT1_CD+_minimamente,AJT1_DD+_minimamente
count,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.00000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.00000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000
mean,3.285714,2.163265,2.285714,3.734694,2.918367,1.816327,2.122449,3.897959,1.816327,1.775510,1.714286,3.632653,2.326531,1.693878,2.204082,3.265306,1.897959,3.551020,2.346939,1.857143,2.224490,2.755102,3.020408,3.571429,2.081633,3.000000,2.040816,2.448980,2.571429,3.367347,3.224490,3.428571,2.244898,3.142857,2.714286,3.734694,1.387755,1.938776,2.857143,3.714286,1.428571,1.632653,2.571429,3.591837,2.612245,2.959184,2.938776,3.775510,1.346939,1.448980,3.142857,3.632653,2.734694,3.040816,2.959184,3.448980,1.489796,2.530612,2.63

In [26]:
natives_ajt_df = natives_df[filter_cols]
natives_ajt_df.describe().to_excel(writer, sheet_name="natives_overview", index=True)
natives_ajt_df.describe()

,AJT0_AA-_aqui,AJT0_BA-_aqui,AJT0_CA-_aqui,AJT0_DA-_aqui,AJT0_AA-_afuera,AJT0_BA-_afuera,AJT0_CA-_afuera,AJT0_DA-_afuera,AJT0_AA-_cerca,AJT0_BA-_cerca,AJT0_CA-_cerca,AJT0_DA-_cerca,AJT0_AA-_lejos,AJT0_BA-_lejos,AJT0_CA-_lejos,AJT0_DA-_lejos,AJT0_AB-_casi,AJT0_BB-_casi,AJT0_CB-_casi,AJT0_DB-_casi,AJT0_AB+_aproximadamente,AJT0_BB+_aproximadamente,AJT0_CB+_aproximadamente,AJT0_DB+_aproximadamente,AJT0_AB-_apenas,AJT0_BB-_apenas,AJT0_CB-_apenas,AJT0_DB-_apenas,AJT0_AB+_solamente,AJT0_BB+_solamente,AJT0_CB+_solamente,AJT0_DB+_solamente,AJT0_AC+_facilmente,AJT0_BC+_facilmente,AJT0_CC+_facilmente,AJT0_DC+_facilmente,AJT0_AC-_bien,AJT0_BC-_bien,AJT0_CC-_bien,AJT0_DC-_bien,AJT0_AC-_mal,AJT0_BC-_mal,AJT0_CC-_mal,AJT0_DC-_mal,AJT0_AC+_precisamente,AJT0_BC+_precisamente,AJT0_CC+_precisamente,AJT0_DC+_precisamente,AJT0_AD-_mucho,AJT0_BD-_mucho,AJT0_CD-_mucho,AJT0_DD-_mucho,AJT0_AD+_mayormente,AJT0_BD+_mayormente,AJT0_CD+_mayormente,AJT0_DD+_mayormente,AJT0_AD-_poco,AJT0_BD-_poco,AJT0_CD-_poco,AJT0_DD-_poco,AJT0_AD+_minimamente,AJT0_BD+_minimamente,AJT0_CD+_minimamente,AJT0_DD+_minimamente,AJT1_AA-_aqui,AJT1_BA-_aqui,AJT1_CA-_aqui,AJT1_DA-_aqui,AJT1_AA-_afuera,AJT1_BA-_afuera,AJT1_CA-_afuera,AJT1_DA-_afuera,AJT1_AA-_cerca,AJT1_BA-_cerca,AJT1_CA-_cerca,AJT1_DA-_cerca,AJT1_AA-_lejos,AJT1_BA-_lejos,AJT1_CA-_lejos,AJT1_DA-_lejos,AJT1_AB-_casi,AJT1_BB-_casi,AJT1_CB-_casi,AJT1_DB-_casi,AJT1_AB+_aproximadamente,AJT1_BB+_aproximadamente,AJT1_CB+_aproximadamente,AJT1_DB+_aproximadamente,AJT1_AB-_apenas,AJT1_BB-_apenas,AJT1_CB-_apenas,AJT1_DB-_apenas,AJT1_AB+_solamente,AJT1_BB+_solamente,AJT1_CB+_solamente,AJT1_DB+_solamente,AJT1_AC+_facilmente,AJT1_BC+_facilmente,AJT1_CC+_facilmente,AJT1_DC+_facilmente,AJT1_AC-_bien,AJT1_BC-_bien,AJT1_CC-_bien,AJT1_DC-_bien,AJT1_AC-_mal,AJT1_BC-_mal,AJT1_CC-_mal,AJT1_DC-_mal,AJT1_AC+_precisamente,AJT1_BC+_precisamente,AJT1_CC+_precisamente,AJT1_DC+_precisamente,AJT1_AD-_mucho,AJT1_BD-_mucho,AJT1_CD-_mucho,AJT1_DD-_mucho,AJT1_AD+_mayormente,AJT1_BD+_mayormente,AJT1_CD+_mayormente,AJT1_DD+_mayormente,AJT1_AD-_poco,AJT1_BD-_poco,AJT1_CD-_poco,AJT1_DD-_poco,AJT1_AD+_minimamente,AJT1_BD+_minimamente,AJT1_CD+_minimamente,AJT1_DD+_minimamente
count,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.0,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.0,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.0,19.000000,19.000000,19.000000,19.000000,19.000000,19.0,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.0,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000
mean,3.105263,2.473684,3.105263,3.315789,2.947368,2.052632,2.789474,3.842105,1.368421,1.315789,2.000000,3.842105,1.684211,1.473684,1.684211,2.263158,1.684211,4.0,2.210526,1.368421,1.578947,2.000000,3.789474,3.315789,1.947368,3.578947,2.736842,2.052632,3.052632,3.842105,4.0,3.368421,2.157895,3.421053,3.842105,3.842105,1.105263,1.684211,3.947368,3.473684,1.368421,1.421053,4.0,3.631579,2.157895,2.368421,3.210526,3.263158,1.0,1.263158,3.842105,2.368421,2.315789,3.105263,3.315789,2.842105,1.105263,1.578947,3.157895,2.368421,2.000000,2.263158,3.157895,2.

In [27]:
# I want to do stats on the data now, so I am going to make a new table which has headers of PID, 
# Group (NS, Beginner, Intermediate, Advanced), question ID, and the rating
new_df = finished_df

def label_group(row):
    if not pd.isna(row["N1"]):
        return "Native"
    if int(row['SC0']) < 30:
        return "Beginner"
    if int(row['SC0']) < 40:
        return "Intermediate"
    return "Advanced"
    
new_df["Group"] = df.apply(lambda row: label_group(row), axis=1)

def get_adverb_class(qid):
    type_key = {
        "A": "Place",
        "B": "Extent",
        "C": "Manner",
        "D": "Quantity"
    }
    
    return type_key[qid.split("_")[1][1]]

def get_structure(qid):
    structure_key = qid.split("_")[1][0]
    
    d = {
        "A": "AdvSVO",
        "B": "SAdvVO",
        "C": "SVAdvO",
        "D": "SVOAdv"
    }
    
    return d[structure_key]

headers = {
    "PID": [],
    "Group": [],
    "QID": [],
    "Rating": [],
    "Adverb": [],
    "Mente": [],
    "Adverb_Class": [],
    "Structure": []
}

# Now, we go through each person and make a row for each question, PID combination
for pid, row in new_df.iterrows():
    group = row["Group"]
    
    for col in new_df.columns:
        if "AJT" in col:
            qid = col
            mente = 'Yes' if qid.split("_")[1][-1] == "+" else 'No'
            adverb = qid.split("_")[-1]
            
            rating = row[col]
            # Add the current data as a row to the stats df
            
            headers["PID"].append(pid)
            headers["Group"].append(group)
            headers["QID"].append(qid)
            headers["Rating"].append(rating)
            headers["Adverb"].append(adverb)
            headers["Mente"].append(mente)
            headers["Adverb_Class"].append(get_adverb_class(qid))
            headers["Structure"].append(get_structure(qid))
            

df = pd.DataFrame.from_dict(headers)
df.to_csv('ajt_data.csv', index=False)

# These make the ordering of the output nice
df['Group'] = pd.Categorical(df['Group'], ["Beginner", "Intermediate", "Advanced", "Native"])
df['Adverb'] = pd.Categorical(df['Adverb'], ['aqui', 'afuera', 'cerca', 'lejos', 
                                             'casi', 'aproximadamente', 'apenas', 'solamente', 
                                             'facilmente', 'bien', 'mal', 'precisamente', 
                                             'mucho', 'mayormente', 'poco', 'minimamente'])
df['Adverb_Class'] = pd.Categorical(df['Adverb_Class'], ['Place', 'Extent', 'Manner', 'Quantity'])
df.head()

,PID,Group,QID,Rating,Adverb,Mente,Adverb_Class,Structure
0,4,Intermediate,AJT0_AA-_aqui,4,aqui,No,Place,AdvSVO
1,4,Intermediate,AJT0_BA-_aqui,1,aqui,No,Place,SAdvVO
2,4,Intermediate,AJT0_CA-_aqui,3,aqui,No,Place,SVAdvO
3,4,Intermediate,AJT0_DA-_aqui,4,aqui,No,Place,SVOAdv
4,4,Intermediate,AJT0_AA-_afuera,4,afuera,No,Place,AdvSVO


In [28]:
# Get the number of participants for each group
df.groupby(['Group']).agg('count')['PID'] / df['QID'].nunique()

Group
Beginner        29.0
Intermediate    13.0
Advanced         7.0
Native          19.0
Name: PID, dtype: float64

In [29]:
# Get the averages for each group
ratings_df = pd.pivot_table(df, values="Rating", index="Group")
ratings_df.to_excel(writer, sheet_name='Overall Ratings')
ratings_df

,Rating
Group,
Beginner,2.605065
Intermediate,2.668870
Advanced,2.735491
Native,2.543174


In [30]:
# Get the average for each adverb
adv_ratings_df = pd.pivot_table(df, values="Rating", index=["Group", "Adverb"]).unstack()
adv_ratings_df.to_excel(writer, sheet_name='Adverb Ratings')
adv_ratings_df

Rating                                          \
Adverb            aqui    afuera     cerca     lejos      casi   
Group                                                            
Beginner      2.784483  2.650862  2.362069  2.275862  2.370690   
Intermediate  2.942308  2.644231  2.173077  2.278846  2.384615   
Advanced      3.196429  2.910714  2.410714  2.410714  2.053571   
Native        3.019737  2.894737  2.006579  1.776316  2.217105   

                                                                       \
Adverb       aproximadamente    apenas solamente facilmente      bien   
Group                                                                   
Beginner            2.862069  2.431034  3.056034   2.849138  2.435345   
Intermediate        3.134615  2.192308  3.153846   3.134615  2.451923   
Advanced            2.892857  2.285714  3.303571   3.214286  2.678571   
Native              2.572368  2.671053  3.421053   3.388158  2.532895   

                                                                                
Adverb             mal precisamente     mucho mayormente      poco minimamente  
Group                                                                           
Beginner      2.185345     3.060345  2.357759   2.900862  2.431034    2.668103  
Intermediate  2.163462     3.201923  2.384615   3.153846  2.480769    2.826923  
Advanced      2.410714     3.053571  2.375000   3.232143  2.571429    2.767857  
Native        2.309211     2.717105  1.953947   2.921053  1.921053    2.368421

In [31]:
# Split these into each structure
import math
adv_ratings_per_struct_df = pd.pivot_table(df, values="Rating", index=["Adverb", "Group", "Structure"]).unstack(level=2)


adv_ratings_per_struct_df = df.groupby(["Adverb", "Group", "Structure"]).agg(['mean', 'count', 'std'])["Rating"]

ci_95_variance = []

for index, row in adv_ratings_per_struct_df.iterrows():
    mean, count, std = (row["mean"], row['count'], row['std'])
    
    ci_95_variance.append(1.96*std/math.sqrt(count))

adv_ratings_per_struct_df['ci95_variance'] = ci_95_variance
adv_ratings_per_struct_df.drop(columns=['count', 'std'], inplace=True)
adv_ratings_per_struct_df = adv_ratings_per_struct_df.unstack(level=2)

adv_ratings_per_struct_df.to_excel(writer, sheet_name='Adverb Rating Per Struct')
adv_ratings_per_struct_df

mean                                \
Structure                   AdvSVO    SAdvVO    SVAdvO    SVOAdv   
Adverb      Group                                                  
aqui        Beginner      3.344828  2.189655  1.862069  3.741379   
            Intermediate  3.500000  2.230769  2.230769  3.807692   
            Advanced      3.571429  2.642857  2.642857  3.928571   
            Native        3.342105  2.473684  2.736842  3.526316   
afuera      Beginner      3.017241  1.724138  2.000000  3.862069   
...                            ...       ...       ...       ...   
poco        Native        1.184211  1.605263  2.763158  2.131579   
minimamente Beginner      2.206897  2.586207  2.293103  3.586207   
            Intermediate  2.346154  3.076923  2.576923  3.307692   
            Advanced      2.000000  2.214286  3.285714  3.571429   
            Native        1.842105  2.184211  3.026316  2.421053   

                         ci95_variance                                
Structure                       AdvSVO    SAdvVO    SVAdvO    SVOAdv  
Adverb      Group                                                     
aqui        Beginner          0.243686  0.252607  0.247924  0.149058  
            Intermediate      0.312278  0.349056  0.331692  0.154492  
            Advanced          0.395980  0.441013  0.390227  0.140000  
            Native            0.279310  0.319983  0.359756  0.230668  
afuera      Beginner          0.302950  0.235150  0.250496  0.089520  
...                                ...       ...       ...       ...  
poco        Native            0.162886  0.281873  0.365408  0.378172  
minimamente Beginner          0.275920  0.250656  0.264084  0.193040  
            Intermediate      0.405959  0.324767  0.394601  0.356785  
            Advanced          0.459431  0.510716  0.380445  0.338503  
            Native            0.281745  0.284917  0.309125  0.343813  

[64 rows x 8 columns]

In [32]:
# Get the averages for each adverb type
adv_class_rating_df = df.groupby(["Group", "Adverb_Class"]).agg(['mean', 'count', 'std'])["Rating"]

ci_95_variance = []

for index, row in adv_class_rating_df.iterrows():
    mean, count, std = (row["mean"], row['count'], row['std'])
    
    ci_95_variance.append(1.96*std/math.sqrt(count))

adv_class_rating_df['ci95_variance'] = ci_95_variance
adv_class_rating_df.drop(columns=['count', 'std'], inplace=True)
adv_class_rating_df = adv_class_rating_df.unstack()

adv_class_rating_df.to_excel(writer, sheet_name='Adverb Class Ratings')
adv_class_rating_df.to_clipboard()
adv_class_rating_df

mean                               ci95_variance            \
Adverb_Class     Place    Extent    Manner  Quantity         Place    Extent   
Group                                                                          
Beginner      2.518319  2.679957  2.632543  2.589440      0.075713  0.071723   
Intermediate  2.509615  2.716346  2.737981  2.711538      0.107150  0.107624   
Advanced      2.732143  2.633929  2.839286  2.736607      0.141929  0.152899   
Native        2.424342  2.720395  2.736842  2.291118      0.093759  0.095079   

                                  
Adverb_Class    Manner  Quantity  
Group                             
Beginner      0.078037  0.074251  
Intermediate  0.116357  0.106674  
Advanced      0.154458  0.154270  
Native        0.096245  0.093256

In [33]:
adv_class_rating_per_struct_df = df.groupby(["Adverb_Class", "Group", "Structure"]).agg(['mean', 'count', 'std'])["Rating"]

ci_95_variance = []

for index, row in adv_class_rating_per_struct_df.iterrows():
    mean, count, std = (row["mean"], row['count'], row['std'])
    
    ci_95_variance.append(1.96*std/math.sqrt(count))

adv_class_rating_per_struct_df['ci95_variance'] = ci_95_variance
adv_class_rating_per_struct_df.drop(columns=['count', 'std'], inplace=True)
adv_class_rating_per_struct_df = adv_class_rating_per_struct_df.unstack()

adv_class_rating_per_struct_df.to_excel(writer, sheet_name='Adverb Class Rating Per Struct')
adv_class_rating_per_struct_df

mean                                \
Structure                    AdvSVO    SAdvVO    SVAdvO    SVOAdv   
Adverb_Class Group                                                  
Place        Beginner      2.719828  1.862069  1.961207  3.530172   
             Intermediate  2.653846  1.788462  2.086538  3.509615   
             Advanced      2.910714  1.821429  2.428571  3.767857   
             Native        2.342105  1.809211  2.342105  3.203947   
Extent       Beginner      2.250000  2.918103  2.547414  3.004310   
             Intermediate  2.028846  3.394231  2.846154  2.596154   
             Advanced      1.946429  3.303571  2.839286  2.446429   
             Native        2.019737  3.230263  3.151316  2.480263   
Manner       Beginner      1.991379  2.366379  2.508621  3.663793   
             Intermediate  1.759615  2.653846  3.038462  3.500000   
             Advanced      1.803571  2.285714  3.571429  3.696429   
             Native        1.750000  2.236842  3.440789  3.519737   
Quantity     Beginner      1.952586  2.461207  2.512931  3.431034   
             Intermediate  2.048077  2.663462  2.798077  3.336538   
             Advanced      1.875000  2.250000  3.232143  3.589286   
             Native        1.611842  2.013158  3.019737  2.519737   

                          ci95_variance                                
Structure                        AdvSVO    SAdvVO    SVAdvO    SVOAdv  
Adverb_Class Group                                                     
Place        Beginner          0.153986  0.117189  0.122882  0.097195  
             Intermediate      0.210485  0.153084  0.173793  0.151488  
             Advanced          0.283753  0.206578  0.205063  0.141188  
             Native            0.186680  0.142585  0.181220  0.167026  
Extent       Beginner          0.134802  0.135846  0.143052  0.139376  
             Intermediate      0.193971  0.161620  0.200001  0.216780  
             Advanced          0.256720  0.254279  0.281547  0.315565  
             Native            0.167158  0.166154  0.175262  0.183848  
Manner       Beginner          0.147615  0.150471  0.139629  0.088707  
             Intermediate      0.197704  0.231578  0.203827  0.151499  
             Advanced          0.261533  0.290003  0.171983  0.140556  
             Native            0.150040  0.179020  0.151686  0.109732  
Quantity     Beginner          0.133460  0.144093  0.136509  0.111327  
             Intermediate      0.197494  0.211914  0.193829  0.174821  
             Advanced          0.278365  0.292298  0.234217  0.178528  
             Native            0.139399  0.171621  0.174985  0.178301

In [34]:
# Look at mente overall
mente_df = df.groupby(["Group", "Mente"]).agg(['mean', 'count', 'std'])["Rating"]

ci_95_variance = []

for index, row in mente_df.iterrows():
    mean, count, std = (row["mean"], row['count'], row['std'])
    
    ci_95_variance.append(1.96*std/math.sqrt(count))

mente_df['ci95_variance'] = ci_95_variance
mente_df.drop(columns=['count', 'std'], inplace=True)
mente_df = mente_df.unstack()

mente_df.to_excel(writer, sheet_name='Mente')
mente_df

mean           ci95_variance          
Mente               No       Yes            No       Yes
Group                                                   
Beginner      2.428448  2.899425      0.047742  0.057422
Intermediate  2.409615  3.100962      0.070487  0.076139
Advanced      2.530357  3.077381      0.097302  0.110225
Native        2.330263  2.898026      0.061810  0.069595

In [35]:
# Split by structure
mente_in_structs_df = pd.pivot_table(df, values="Rating", index=["Mente", "Group", "Structure"]).unstack(level=1)
mente_in_structs_df.to_excel(writer, sheet_name='Mente in Structs')
mente_in_structs_df

Rating                                 
Group            Beginner Intermediate  Advanced    Native
Mente Structure                                           
No    AdvSVO     2.105172     1.919231  2.035714  1.781579
      SAdvVO     2.100000     2.161538  2.107143  1.997368
      SVAdvO     2.203448     2.423077  2.707143  2.710526
      SVOAdv     3.305172     3.134615  3.271429  2.831579
Yes   AdvSVO     2.433908     2.461538  2.297619  2.179825
      SAdvVO     2.905172     3.397436  2.928571  2.864035
      SVAdvO     2.681034     3.141026  3.535714  3.451754
      SVOAdv     3.577586     3.403846  3.547619  3.096491

In [36]:
ratings_by_stuct_df = df.groupby(["Structure", "Group"]).agg(['mean', 'count', 'std'])["Rating"]

ci_95_variance = []

for index, row in ratings_by_stuct_df.iterrows():
    mean, count, std = (row["mean"], row['count'], row['std'])
    
    ci_95_variance.append(1.96*std/math.sqrt(count))

ratings_by_stuct_df['ci95_variance'] = ci_95_variance
ratings_by_stuct_df.drop(columns=['count', 'std'], inplace=True)
ratings_by_stuct_df = ratings_by_stuct_df.unstack()
ratings_by_stuct_df.to_excel(writer, sheet_name='Structs')
ratings_by_stuct_df

mean                                  ci95_variance  \
Group      Beginner Intermediate  Advanced    Native      Beginner   
Structure                                                            
AdvSVO     2.228448     2.122596  2.133929  1.930921      0.073919   
SAdvVO     2.401940     2.625000  2.415179  2.322368      0.072747   
SVAdvO     2.382543     2.692308  3.017857  2.988487      0.069550   
SVOAdv     3.407328     3.235577  3.375000  2.930921      0.057564   

                                            
Group     Intermediate  Advanced    Native  
Structure                                   
AdvSVO        0.104500  0.146739  0.083705  
SAdvVO        0.110377  0.148925  0.093219  
SVAdvO        0.102334  0.125846  0.091193  
SVOAdv        0.094687  0.124794  0.088422

In [37]:
writer.save()